In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.fft import fft,fft2, fftshift
from skimage.io import imread
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

class CustomDataset(Dataset):
    def __init__(self, img_folder, csv_file,transform=None):
        """
        Args:
            img_folder (str): Path to the folder containing images.
            csv_file (str): Path to the CSV file with image names and labels.
            transform (callable, optional): Optional transform to be applied on the image.
        """
        self.img_folder = img_folder
        self.df = pd.read_csv(csv_file)
        self.transform = transform

        # Extract image file names and labels from the CSV file
        self.img_files = self.df['Image'].values
        self.labels = self.df['any'].values

    def __len__(self):
        # Return the length of the dataset (number of samples)
        return len(self.img_files)
    def __getitem__(self, index):
    # Load the image file
        img_filename = self.img_files[index]
        img_path = os.path.join(self.img_folder, f"{img_filename}.npy")  # Construct the .npy file path

    # Load the image as a NumPy array from the .npy file
        image = np.load(img_path)
        num_angles = image.shape[0]
        num_channels = image.shape[2]
        for angle_index in range(num_angles):
                # Iterate over each channel
            for channel_index in range(num_channels):
                    # Apply Fourier transform to the current channel at the current angle
                image[angle_index, :, channel_index] = abs(fft(image[angle_index, :, channel_index]))

    # Convert the NumPy array to a PyTorch tensor

        image = torch.from_numpy(image).float()
        image=image.permute(2,0,1)

    # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

    # Get the label
        label = self.labels[index]

    # Convert the label to a PyTorch tensor (if necessary)
        #label = torch.tensor(label, dtype=torch.float32)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

        if self.transform:
            image = self.transform(image)



In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler


train_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
train_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/train_x.csv'  # Replace with your CSV file path
train_dataset = CustomDataset(train_img_folder, train_csv_file)

test_img_folder ='/mnt/DATA/EE21B040/Downloads/sinograms_3W_test' # Replace with your image folder path
test_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/test_x.csv' # Replace with your CSV file path
test_dataset = CustomDataset(test_img_folder, test_csv_file)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=True,drop_last=True)

# Set the random seed for reproducibility (optional)
torch.manual_seed(42)

# Get the length of your train_dataset
dataset_length = len(train_dataset)

# Generate a list of indices from 0 to dataset_length
indices = list(range(dataset_length))

# Shuffle the indices
torch.random.manual_seed(42)  # Set random seed for reproducibility
# torch.random.shuffle(indices)
indices = torch.randperm(dataset_length)
indices = indices.tolist()
# Select 20,000 random indices
selected_indices = indices[:30000]

# Create a SubsetRandomSampler using the selected indices
subset_sampler = SubsetRandomSampler(selected_indices)

train_loader = DataLoader(train_dataset, batch_size=50, sampler=subset_sampler,drop_last=True)
valid_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
valid_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/valid_x.csv' # Replace with your CSV file path
valid_dataset = CustomDataset(valid_img_folder, valid_csv_file)
valid_loader = DataLoader(valid_dataset, batch_size=50, shuffle=True,drop_last=True)

In [ ]:
class BinaryClassificationCNN(nn.Module):
    def __init__(self):
        super(BinaryClassificationCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 45 * 45, 256)
        self.fc2 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 45 * 45)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import time
learning_rate=0.0001
num_epochs=15
model = BinaryClassificationCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the ModelCheckpoint function to save the model every 10 epochs
def save_checkpoint(epoch):
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')

# Training loop
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = (outputs >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(correct / total)

    # Validation loop
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_losses.append(valid_loss / len(valid_loader))
    valid_accuracies.append(correct / total)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.4f}, '
          f'Valid Loss: {valid_losses[-1]:.4f}, Valid Acc: {valid_accuracies[-1]:.4f}')

    # Save the model every 10 epochs
    if (epoch + 1) % 10 == 0:
        save_checkpoint(epoch + 1)

In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predicted = (outputs >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = correct / total

print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}')

# Plotting training and validation curves


plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Valid Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()
